# 원판 돌리기

반지름이 1, 2, ..., N인 원판이 크기가 작아지는 순으로 바닥에 놓여있고, 원판의 중심은 모두 같다. 원판의 반지름이 i이면, 그 원판을 i번째 원판이라고 한다. 각각의 원판에는 M개의 정수가 적혀있고, i번째 원판에 적힌 j번째 수의 위치는 (i, j)로 표현한다. 수의 위치는 다음을 만족한다.

- (i, 1)은 (i, 2), (i, M)과 인접하다.
- (i, M)은 (i, M-1), (i, 1)과 인접하다.
- (i, j)는 (i, j-1), (i, j+1)과 인접하다. (2 ≤ j ≤ M-1)
- (1, j)는 (2, j)와 인접하다.
- (N, j)는 (N-1, j)와 인접하다.
- (i, j)는 (i-1, j), (i+1, j)와 인접하다. (2 ≤ i ≤ N-1)  
아래 그림은 N = 3, M = 4인 경우이다.

원판의 회전은 독립적으로 이루어진다. 2번 원판을 회전했을 때, 나머지 원판은 회전하지 않는다. 원판을 회전시킬 때는 수의 위치를 기준으로 하며, 회전시킨 후의 수의 위치는 회전시키기 전과 일치해야 한다.

원판을 아래와 같은 방법으로 총 T번 회전시키려고 한다. 원판의 회전 방법은 미리 정해져 있고, i번째 회전할때 사용하는 변수는 xi, di, ki이다.

1. 번호가 xi의 배수인 원판을 di방향으로 ki칸 회전시킨다. di가 0인 경우는 시계 방향, 1인 경우는 반시계 방향이다.
2. 원판에 수가 남아 있으면, 인접하면서 수가 같은 것을 모두 찾는다.
    1. 그러한 수가 있는 경우에는 원판에서 인접하면서 같은 수를 모두 지운다.
    2. 없는 경우에는 원판에 적힌 수의 평균을 구하고, 평균보다 큰 수에서 1을 빼고, 작은 수에는 1을 더한다.
원판을 T번 회전시킨 후 원판에 적힌 수의 합을 구해보자.

첫째 줄에 N, M, T이 주어진다.

둘째 줄부터 N개의 줄에 원판에 적힌 수가 주어진다. i번째 줄의 j번째 수는 (i, j)에 적힌 수를 의미한다.

다음 T개의 줄에 xi, di, ki가 주어진다.

원판을 T번 회전시킨 후 원판에 적힌 수의 합을 출력한다.

In [1]:
from collections import deque

# 가장 큰 반지름 n, 각 원판에 적혀있는 숫자의 갯수 m, 회전하는 횟수 t
n, m, t = map(int, input().split())

# 각 원판에 적혀있는 수
board = [list(map(int, input().split())) for _ in range(n)]

# 동 - 서 - 남 - 북
dx = [0, 0, 1, -1]
dy = [1, -1, 0, 0]

def rotate(x, d, k):
    q = deque()
    q.extend(board[x])
    
    # 시계방향일 경우
    if d == 0:
        # 시계방향으로 k배 만큼 돌리기
        q.rotate(k)
    else:
        # 반시계방향 만큼 k배 돌리기
        q.rotate(-k)
        
    board[x] = list(q)
    
def change_avg():
    avg_count = 0
    all_sum = 0
    
    for i in range(n):
        for j in range(m):
            if board[i][j] != 0:
                avg_count += 1
                all_sum += board[i][j]
    
    if avg_count == 0:
        return False
    
    avg = all_sum / avg_count
    
    for i in range(n):
        for j in range(m):
            if board[i][j] != 0:
                if board[i][j] > avg:
                    board[i][j] -= 1
                elif board[i][j] < avg:
                    board[i][j] += 1
    
    return True

def solve(x, y):
    q = deque()
    q.append((x,y))
    visit[x][y] = True
    value = board[x][y]
    board[x][y] = 0
    count = 0
    
    while q:
        x, y = q.popleft()
        
        for i in range(4):
            nx = x + dx[i]
            # 해당 부분은 원형으로 돌기 때문에 나머지를 통해 순환
            ny = (y + dy[i]) % m
            
            if 0 <= nx < n and 0 <= ny < m:
                # 방문하지 않은 인접한 부분 중에서
                if not visit[nx][ny]:
                    # 서로 같은 값을 가질 경우
                    if board[nx][ny] == value:
                        count += 1
                        # 값을 0으로 바꿈
                        board[nx][ny] = 0
                        visit[nx][ny] = True
                        q.append((nx, ny))
    
    # 인접한 경우에 대해 같은 값이 없을 경우에는 원래 값 넣어주는 방식
    if count == 0:
        board[x][y] = value
        
    return count

for _ in range(t):
    x, d, k = map(int, input().split())
    check_sum = 0
    
    for i in range(n):
        check_sum += sum(board[i])
        
        if (i+1) % x == 0:
            rotate(i, d, k)
    
    if check_sum == 0:
        break
    
    else:
        visit = [[False] * m for _ in range(n)]
        count = 0
        # 모든 행과 열에 대하여 BFS를 돌린 뒤, 인접한 숫자 제거
        for i in range(n):
            for j in range(m):
                if not visit[i][j] and board[i][j] != 0:
                    # 인접하면서 같은 수 탐색
                    count += solve(i,j)
        
        # 인접한 숫자가 하나도 없다면, change_avg()를 통해 원판의 숫자 변경
        if count == 0:
            change_avg()
            
answer = 0
for i in range(n):
    answer += sum(board[i])
    
print(answer)

4 4 3
1 1 2 3
5 2 4 2
3 1 3 5
2 1 3 2
2 0 1
3 1 3
2 0 2
22
